In [ ]:
import json
import re
import warnings
from datetime import datetime

import camelot
import pandas as pd

warnings.filterwarnings("ignore")

In [ ]:
# Extract tables from the downloaded PDF
tables = camelot.read_pdf("data/promsobstv1.pdf", pages='all')

# Iterate through each table and export its data
data = tables[0].df
assert type(data) == pd.DataFrame
for i, table in enumerate(tables):
    # drop first two rows because they are headers and empty lines
    table_data = table.df.drop([0,1])
    data = pd.concat([data, table_data], ignore_index=True)

def clean_column_name(col_name):
    col_name = col_name.replace('\n', '')
    col_name = col_name.replace('-','')
    return col_name

data.columns = list(map(clean_column_name, data.iloc[0]))  # Set the first row as header
data = data.drop(0).reset_index(drop=True)  # Remove the first row from

In [ ]:
df_data = data.drop([0, 1]).reset_index(drop=True) 
df_data.columns

Index(['', 'Тип регистрационного документа', '№заявки, патента, АС',
       'Приоритет, регистрация', 'Название', 'статус документа',
       'Заявитель, правообладатель (обладатель имущественных прав)',
       'Публикация (состояние делопроизводства)', 'Авторы:', '', '', '', '',
       '', '', '', '', '', '', ''],
      dtype='object')

In [ ]:
names = data['Название']
# find dates in format dd.mm.yyyy
dates = re.compile(r'\d{2}\.\d{2}\.\d{4}')
dates = [date for date in names.str.findall(dates)]

# separate names and dates
names = [name.replace(date[0], '').strip() if len(date) > 0 else name.strip() for name, date in zip(names[2:], dates[2:])]
names = list(map(lambda x: re.sub(r'\s+', ' ', x), names))
dates = list(map(lambda x: datetime.strptime(x[0], '%d.%m.%Y').strftime('%Y-%m-%d %H:%M:%S') if len(x) > 0 else None, dates[2:]))

df_data['Приоритет, регистрация'] = dates
df_data['Название'] = names

df_data = df_data.map(str).replace('\n', ' ', regex=True)
df_data = df_data.map(str).replace(r'\s+', ' ', regex=True)

df_data['Тип регистрационного документа'] = df_data['Тип регистрационного документа'].replace({'3аявка на изобрет ение': '3аявка на изобретение', 'Патент Междун ародна я заявка': 'Патент, Международная заявка'})

In [ ]:
# авторы и публикация не имеют отношения к предмету патента,
# поэтому они отбрасываются
columns = df_data.columns[1:7]
cleaned_data = df_data[columns]
cleaned_data

,Тип регистрационного документа,"№заявки, патента, АС","Приоритет, регистрация",Название,статус документа,"Заявитель, правообладатель (обладатель имущественных прав)"
0,3аявка на изобретение,(RU)2025122356,2025-08-14 00:00:00,ХЕМО-ЭЛЕКТРОННЫЙ КОНВЕРТЕР НА ОСНОВЕ НАНОПОРОШ...,Заявка,ОИЯИ
1,3аявка на изобретение,(RU)2025117067,2025-06-20 00:00:00,СПОСОБ И СИСТЕМА ИНТЕЛЛЕКТУАЛЬНОГО САМООРГАНИЗ...,Заявка,ОИЯИ
2,3аявка на изобретение,(RU)2025105865,2025-03-21 00:00:00,СИТУАЦИЯХ СПОСОБ ИЗМЕРЕНИЯ ВРЕМЕНИ ЖИЗНИ ВОЗБУ...,Заявка,ОИЯИ
3,3аявка на изобретение,(RU)2024133540,2025-03-13 00:00:00,МНОГОФУНКЦИОНАЛЬНОЕ УСТРОЙСТВО ТАНГЕНЦИАЛЬНОЙ ...,Заявка,ОИЯИ
4,3аявка на изобретение,(RU)2024133540,2024-10-08 00:00:00,СПОСОБ ПОЛУЧЕНИЯ НЕЙТРОНОВ ПРИ ПОМОЩИ ИМПУЛЬСН...,Заявка,ОИЯИ
...,...,...,...,...,...,...
311,АС,(SU) 1075937,1982-08-19 00:00:00,ВЗРЫВОЭМИССИОННЫЙ КАТОД,не действует,СССР
312,АС,(SU) 884521,1982-03-10 00:00:00,УСКОРИТЕЛЬ ЭЛЕКТРОНОВ,не действует,СССР
313,АС,(SU) 745271,1980-07-10 00:00:00,МУЛЬТИПЛЕКСНЫЙ СКВИД,не публ.,ОИЯИ
314,АС,(DE) 19549417,1979-01-08 00:00:00,СПОСОБ ПОЛУЧЕНИЯ ГОЛОГРАММ БЕЗ ОПОРНОГО ПУЧКА,не действует,СССР


In [239]:
cleaned_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 316 entries, 0 to 315
Data columns (total 6 columns):
 #   Column                                                      Non-Null Count  Dtype 
---  ------                                                      --------------  ----- 
 0   Тип регистрационного документа                              316 non-null    object
 1   №заявки, патента, АС                                        316 non-null    object
 2   Приоритет, регистрация                                      316 non-null    object
 3   Название                                                    316 non-null    object
 4   статус документа                                            316 non-null    object
 5   Заявитель, правообладатель (обладатель имущественных прав)  316 non-null    object
dtypes: object(6)
memory usage: 14.9+ KB


In [ ]:
with open("data/extracted_data.json", "w", encoding="utf-8") as f:
    json.dump(cleaned_data.to_dict(orient="records"), f, ensure_ascii=False, indent=4)